In [96]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.metrics import accuracy_score


from featexp import get_trend_stats
from matplotlib import pyplot
from xgboost import plot_importance
from sklearn.feature_selection import SelectFromModel
from numpy import sort

## Load data

In [97]:
dir_data = "/Users/romulo/Documents/Dataset/Earthquake Damage/"
#dir_data = "Dataset/"
df_x_train = pd.read_csv(dir_data+"train_values.csv",index_col="building_id")
print("df_x_train",df_x_train.shape)
df_y_train = pd.read_csv(dir_data+"train_labels.csv",index_col="building_id")
df_x_test = pd.read_csv(dir_data+"test_values.csv",index_col="building_id")
print("df_x_test",df_x_test.shape)

#df_x_train = df_x_train[:100]
#df_y_train = df_y_train[:100]
#df_x_test = df_x_test[:100]

data_train = df_x_train.merge(df_y_train, how='left', left_index=True, right_index=True)
print("data_train",data_train.shape)


print("len train:",len(data_train))
print("len train:",len(df_x_test))

df_x_train (260601, 38)
df_x_test (86868, 38)
data_train (260601, 39)
len train: 260601
len train: 86868


## Preparate data

In [98]:
# get y train
y_train = data_train['damage_grade'].values

# remove y of data_train
data_train = data_train.drop('damage_grade', 1)

In [99]:
# let's put the train data and test data together to make get_dummies and then divide
df_x_all = data_train.append(df_x_test)
print("df_x_all:",df_x_all.shape)

# get dummies from cat columns
cat_var = [key for key in dict(df_x_all.dtypes) if dict(df_x_all.dtypes)[key] in ['object'] ]
df_x_all = pd.get_dummies(df_x_all, prefix=cat_var, columns=cat_var)

#divide x_train and x_test
x_train = df_x_all.iloc[:len(data_train)]
x_test = df_x_all.iloc[len(data_train):]

print("x_train:",x_train.shape)
print("x_test:",x_test.shape)

df_x_all: (347469, 38)
x_train: (260601, 68)
x_test: (86868, 68)


In [100]:
x_train.shape

(260601, 68)

In [101]:
x_test.shape

(86868, 68)

In [102]:
# clear memory
del df_x_all,df_x_train,df_y_train,data_train,df_x_test

In [88]:
#get x_dev and y_dev (10% from train)
#x_train, x_dev, y_train, y_dev = train_test_split( x_train, y_train, test_size=0.1, random_state=42)

In [89]:
print("len x_train : %d  len y_train: %d " %(len(x_train),len(y_train)) )
print("len x_dev   : %d  len y_dev  : %d " %(len(x_dev),len(y_dev)) )
print("len x_test  : %d" %(len(x_test)) )

len x_train : 234540  len y_train: 234540 
len x_dev   : 26061  len y_dev  : 26061 
len x_test  : 86868


## Features

## Testing

In [108]:
max_depth = 6
n_estimator =  1500

model = xgb.XGBClassifier(n_estimators=n_estimator,max_depth=max_depth)
model.fit(x_train, y_train)


#Eval
y_train_pred = model.predict(x_train)
#y_dev_pred = model.predict(x_dev)


accuracy_train = accuracy_score(y_train, y_train_pred)
#accuracy_dev = accuracy_score(y_dev, y_dev_pred)

#pred_y_pred = model.predict(x_dev)

# dict_report = classification_report(y_dev, pred_y_pred,output_dict=True)
# precision_1 = dict_report["1"]["precision"]
# precision_2 = dict_report["2"]["precision"]
# precision_3 = dict_report["3"]["precision"]

# recall_1 = dict_report["1"]["recall"]
# recall_2 = dict_report["2"]["recall"]
# recall_3 = dict_report["3"]["recall"]

# score_1 = dict_report["1"]["f1-score"]
# score_2 = dict_report["2"]["f1-score"]
# score_3 = dict_report["3"]["f1-score"]


print("\taccuracy_train",accuracy_train)
#print("\taccuracy_dev",accuracy_dev)



	accuracy_train 0.7966354695492343


In [109]:
# thresholds = sort(model.feature_importances_)
# for thresh in thresholds:
#     # select features using threshold
#     selection = SelectFromModel(model, threshold=thresh, prefit=True)
#     select_x_train = selection.transform(x_train)
#     # train model
#     selection_model = xgb.XGBClassifier(n_estimators=n_estimator,max_depth=max_depth)
#     selection_model.fit(select_x_train, y_train)
#     # eval model
#     select_x_test = selection.transform(x_dev)
#     y_pred = selection_model.predict(select_x_test)
#     predictions = [round(value) for value in y_pred]
#     accuracy = accuracy_score(y_dev, predictions)
#     print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_x_train.shape[1], accuracy*100.0))

## TIME TO PREDICT AND SUBMIT

In [110]:
predictions = model.predict(x_test)

In [111]:
submission_format = pd.read_csv(dir_data + 'submission_format.csv', index_col='building_id')

In [112]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [113]:
my_submission.to_csv('XGBClassifier_1500_all_submission.csv')